IMPORTING LIBRARIES

In [10]:
from __future__ import absolute_import,division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
from tensorflow.python.tpu import feature_column_v2 as fc
import tensorflow as tf

SETTING LABELS

In [11]:
CSV_COLUMN_NAMES = ['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

READING DATA FROM CSV WITH MODIFIED COLUMN NAMES

In [12]:
train = pd.read_csv("./datasets/iris_training.csv",names=CSV_COLUMN_NAMES,header=0)
test = pd.read_csv("./datasets/iris_test.csv",names=CSV_COLUMN_NAMES,header=0)
print(train.head())
print("\n",test.head())

   SepalLength  SepalWidth  PetalLength  PetalWidth  Species
0          6.4         2.8          5.6         2.2        2
1          5.0         2.3          3.3         1.0        1
2          4.9         2.5          4.5         1.7        2
3          4.9         3.1          1.5         0.1        0
4          5.7         3.8          1.7         0.3        0

    SepalLength  SepalWidth  PetalLength  PetalWidth  Species
0          5.9         3.0          4.2         1.5        1
1          6.9         3.1          5.4         2.1        2
2          5.1         3.3          1.7         0.5        0
3          6.0         3.4          4.5         1.6        1
4          5.5         2.5          4.0         1.3        1


SEPERATING THE SPECIES COLUMN

In [13]:
train_y=train.pop('Species')
test_y=test.pop('Species')
print(train.head())
print("\n",test.head())
print("\n",train_y.head())
print("\n",test_y.head())

   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.4         2.8          5.6         2.2
1          5.0         2.3          3.3         1.0
2          4.9         2.5          4.5         1.7
3          4.9         3.1          1.5         0.1
4          5.7         3.8          1.7         0.3

    SepalLength  SepalWidth  PetalLength  PetalWidth
0          5.9         3.0          4.2         1.5
1          6.9         3.1          5.4         2.1
2          5.1         3.3          1.7         0.5
3          6.0         3.4          4.5         1.6
4          5.5         2.5          4.0         1.3

 0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

 0    1
1    2
2    0
3    1
4    1
Name: Species, dtype: int64


INPUT FUNCTION

In [14]:
def input_fn(features,labels, training=True,batch_size=256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

FEATURE COLUMN ( Provides the column details to model along the unique data it can contain )

In [15]:
my_feature_column = []
for key in train.keys():
    my_feature_column.append(tf.feature_column.numeric_column(key=key))
print(my_feature_column)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


DNN CLASSIFIER ( Setting up the model structure )

In [16]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_column,
    hidden_units=[30,10],
    n_classes=3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ADITYA\\AppData\\Local\\Temp\\tmp3ldg5zns', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


MODEL TRAINING

In [17]:
classifier.train(
    input_fn=lambda: input_fn(train,train_y,training=True),
    steps=5000
)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\ADITYA\AppData\Local\Temp\tmp3ldg5zns\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.6434798, step = 0
INFO:tensorflow:global_step/sec: 663.838
INFO:tensorflow:loss = 1.2009974, step = 100 (0.151 sec)
INFO:tensorflow:global_step/sec: 1028.89
INFO:tensorflow:loss = 1.0639919, step = 200 (0.098 sec)
I

TESTING IT ON A TEST DATASET

In [18]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test,test_y,training=False)
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-04-05T21:17:03
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ADITYA\AppData\Local\Temp\tmp3ldg5zns\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 1.19618s
INFO:tensorflow:Finished evaluation at 2023-04-05-21:17:04
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.46666667, average_loss = 0.79269814, global_step = 5000, loss = 0.79269814
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\ADITYA\AppData\Local\Temp\tmp3ldg5zns\model.ckpt-5000


PRINTING ACCURACY OF THE MODEL

In [19]:
print('\nTest set accuracy: ',(eval_result['accuracy']*100))


Test set accuracy:  46.666666865348816


INPUT FUNCTION FOR CUSTOM USER INPUT

In [20]:
def input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

ADDING LABELS AND PREDICTION SET ( which contains the probabilities of the input being any one of the flower species)

In [22]:
features=['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict={}

INPUT THE CUSTON USER INPUT

In [25]:
print("Please type numeric values as prompted: ")
for feature in features:
    valid=True
    while valid:
        val = input(feature + ": ")
        if not val.isdigit(): valid = False
    predict[feature] = [float(val)]

Please type numeric values as prompted: 


PREDICTING THE FLOWER SPECIES

In [26]:
predictions = classifier.predict(
    input_fn=lambda: input_fn(predict)
)

DISPLAY THE MAX PROBABILITY LABEL ( Species )

In [27]:
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id],100*probability
    ))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ADITYA\AppData\Local\Temp\tmp3ldg5zns\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Virginica" (89.5%)
